In [1]:
import os
from pprint import pprint
from sqlalchemy import insert, select

import settings # type: ignore
from camtrappy.db.schema import Video, Location, Project
from camtrappy.db.testdb import Session
from camtrappy.io import video, parsing

DATA_DIR = settings.data_dir_real
PROJECT = parsing.Project(projectfolder=None, datafolder=DATA_DIR)

home:      n:\Mitarbeiter\Florian\projects\CamTrapPy
data:      n:\Mitarbeiter\Florian\projects\CamTrapPy\tests\data
notebooks: n:\Mitarbeiter\Florian\projects\CamTrapPy\notebooks
--Setup complete--
2021-07-18 16:34:35,702 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-18 16:34:35,703 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("videos")
2021-07-18 16:34:35,704 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-18 16:34:35,706 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("videos")
2021-07-18 16:34:35,707 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-18 16:34:35,708 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("locations")
2021-07-18 16:34:35,709 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-18 16:34:35,710 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("locations")
2021-07-18 16:34:35,711 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-18 16:34:35,713 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("projects")
2021-07-18 16:34:35

In [2]:
pprint(PROJECT.data(full_path=True))
pprint(PROJECT.data(full_path=False))
pprint(PROJECT.data_flat())

{'n:\\Mitarbeiter\\Florian\\projects\\CamTrapPy\\tests\\data\\real_datasets\\1_Tauberbischofsheim': ['n:\\Mitarbeiter\\Florian\\projects\\CamTrapPy\\tests\\data\\real_datasets\\1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_010534_BEE9.mkv',
                                                                                                     'n:\\Mitarbeiter\\Florian\\projects\\CamTrapPy\\tests\\data\\real_datasets\\1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_011034_B0E3.mkv',
                                                                                                     'n:\\Mitarbeiter\\Florian\\projects\\CamTrapPy\\tests\\data\\real_datasets\\1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_012036_B475.mkv',
                                                                                                     'n:\\Mitarbeiter\\Florian\\projects

In [3]:
# # combine sessionmaker Session, with-statement & begin() to start session, handle exceptions, rollback, commit and close at once
# with Session.begin() as session:
#     session.add(object)

dir(Video)

['__abstract__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__mapper__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__table__',
 '__tablename__',
 '__weakref__',
 '_sa_class_manager',
 '_sa_registry',
 'date_added',
 'id',
 'location',
 'location_id',
 'metadata',
 'path',
 'registry']

Fill db with videos

In [4]:
# videos = video.video_paths(data, ['mkv'])

# new_videos = [Video(path=video) for video in videos]
# with Session.begin() as session:
#     session.add_all(new_videos)

In [2]:
project = Project(name='Testproject')
videos = video.video_paths(data, ['mkv'])

for video_path in videos:
    location_name = video.location_from_path(video_path)
    location = Location(name=location_name, project=project)
    vid = Video(path=video_path, location=location)
    
    

In [11]:
with Session.begin() as session:
    project = Project(name='Testproject')
    session.add(project)

videos = video.video_paths(data, ['mkv'])
locations = [video.location_from_path(vid) for vid in videos]
locations_videos = {location: video for location, video in zip(locations, videos)}
print(locations_videos)

# project = Project(name='Testproject')
# new_locations = [Location(name=location, project=project) for location in locations]
# new_videos = [Video(video=path, location=) for path in videos]

# with Session.begin() as session:
    
#     session.add(project)
    # result = session.query(Project).filter(Project.name == 'Testproject').first()
    # print(result.id, result.name)
    

2021-07-16 20:22:09,628 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-16 20:22:09,630 INFO sqlalchemy.engine.Engine INSERT INTO projects (name, date_finished) VALUES (?, ?)
2021-07-16 20:22:09,630 INFO sqlalchemy.engine.Engine [cached since 4287s ago] ('Testproject', None)
2021-07-16 20:22:10,091 INFO sqlalchemy.engine.Engine COMMIT
{'1_Tauberbischofsheim': '1_Tauberbischofsheim\\20190306\\17\\20190306_171415_5EAF_ACCC8EAE987D\\20190307_01\\20190307_010033_1492.mkv'}
